<a href="https://colab.research.google.com/github/iambusra/complingproject/blob/main/Copy_of_the_ultimate_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Brief Intro**
##### **Ne yapacağız?** 
Google News üzerinden haber çekeceğiz.
<br>
<br>
##### **Nasıl yapacağız?** 
Requests ile sayfaya req atacağız, BeautifulSoup ile html kodu içinde gezeceğiz, Pandas ile de metinleri, başlıkları ve linkleri Excel dosyasına kaydedeceğiz. Sonra bu Excel dosyasının içinde looplar ile dolaşarak linklere gideceğiz, body metinlerini çekeceğiz ve bu metinleri yine dosyaya kaydedeceğiz. 
<br>
<br>
##### **Neden Excel?** 
Temiz, erişilebilir ve kolay diye böyle 1 karar aldım. Datamızı initially incelerken Excel üzerinden bakmak çok daha kolay olur, kodla uğraşmadan çıplak gözle de görebiliriz, annotation yapmak istersek Google Sheets üzerinden paylaşıp annotation yaparız, ayrı ayrı çektiğimiz dataları kolayca birleştirebiliriz vs. Gerekince/gerekirse metinleri Excel'den alıp txt'ye aktarmak da çok kolay. 

## Part 1: Lazım paketler

**Lazım şeyleri import edelim.** Bunları önce pip ile bilgisayarınıza install etmeniz gerekecek muhtemelen. Pip komutlarını da yazdım, uncomment edip bir kere çalıştırdıktan sonra tekrar comment out ediniz. Sorun olursa lmk. 

In [ ]:
!pip install requests
import requests  
#html requestleri atabilmemiz için lazım. 

!pip install beautifulsoup4
from bs4 import BeautifulSoup as bs 
#actual scraping işleri için lazım

!pip install pandas
!pip install openpyxl
!pip install xlsxwriter
!pip install xlrd
!pip install numpy
import pandas as pd
import openpyxl
import xlsxwriter
import xlrd
import numpy
#excel işleri için lazım bu iki paket


## Part 2: Sayfaya kısa bir bakış

##### **Link meselesi.**
Tarih aralığı bölüşürüz demiştim ama Google buna izin vermiyor lol. Son x saat, son x gün, son x hafta ve son x ay şeklinde arayabiliyoruz, spesifik tarih aralığı giremiyoruz. Birkaç gün üst üste parametreyi son 24 saat yaparak scrape edebiliriz. Bunun için günleri paylaşmamız gerekir. Grupta konuşalım.

##### **Sayfanın html contentine bi bakalım.**
Bize başlık + link lazım, bunların html tagleri neymiş ona göz atacağız.
<br>
<br>
not: 100 haber başlığı çekeceğiz, max scroll bu kadar. 


In [ ]:
r = requests.get ("https://news.google.com/rss/search?q=kadin+surucu&ceid=TR:tr&hl=tr&gl=TR&when=24")
# linkteki sayfanın html içeriğini çeken kod parçası. requests paketini kullandık.

soup = bs(r.content)
# beautiful soup objectine çevirdik ki o paketi kullanarak html kodu içinde gezebilelim,
# istediğimiz kısımları çekebilelim vs.

print(soup.prettify())
# soup object olarak tanımladığımız şeyi print ediyoruz, bakalım sayfa olması gerektiği
# şekilde gelmiş mi. prettify'ı şu sebeple kullanıyoruz, html kodları aslında matruşka gibi 
# nested structure'lar. direkt bu kodu çektiğimizde indentation gibi nestedness'ı
# daha iyi anlamamıza yardımcı olan formattingler kayboluyor. prettify dediğimizde indentation
# falan yaparak kodu okuması kolay bir hale getiriyoruz.
# 100 haber çektiğimiz için print gigantic olacak. kodu birkaç kere çalıştırmanız gerekirse
# kalabalık yapmasın diye bu print kısmını comment out edebilirsiniz, aşağıdaki kodlarda
# buraya bi dependency yok.


<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<html>
 <body>
  <rss version="2.0" xmlns:media="http://search.yahoo.com/mrss/">
   <channel>
    <generator>
     NFE/5.0
    </generator>
    <title>
     "kadin surucu" - Google Haberler
    </title>
    <link/>
    https://news.google.com/search?q=kadin+surucu&amp;ceid=TR:tr&amp;hl=tr&amp;gl=TR&amp;when=24
    <language>
     tr
    </language>
    <webmaster>
     news-webmaster@google.com
    </webmaster>
    <copyright>
     2021 Google Inc.
    </copyright>
    <lastbuilddate>
     Thu, 27 May 2021 22:05:59 GMT
    </lastbuilddate>
    <description>
     Google Haberler
    </description>
    <item>
     <title>
      Kadın sürücü aracıyla bariyerlere çarparak yaralandı - Haberler.com
     </title>
     <link/>
     https://www.haberler.com/kadin-surucu-bariyerlere-carparak-yaralandi-14160750-haberi/
     <guid ispermalink="false">
      CBMiVWh0dHBzOi8vd3d3LmhhYmVybGVyLmNvbS9rYWRpbi1zdXJ1Y3UtYmFyaXllcmxlcmUtY2FycGFyYWsteW

**Esas scraping kısmı başlıyor.** Yukarıda gördük ki `<title>` 'da haberin başlığı var, `<link/>` 'te ve `<description>` 'da da linkler var. Bizim sayfadan bu kısımları almamız lazım. 

 html'de etiket formatı şöyle:


```
<etiket> #burada etiket başlıyor.
content
</etiket> #burada etiket bitiyor.
```

`<link/>` kısmında etiket temiz halde değil. CSS ve html çok bilmediğim için neden böyle bir stil tercih ettiklerini net açıklayamayacağım, prolly Google'ın kaynak sitelerden veri çekme processiyle ilgili. So, `<link/>` etiketi yamuk olduğu için onun yerine `<description>` 'ı kullanacağız fakat bu da bize bir miktar unnecessary veri getirecek, çok tidy olmayacak. o yüzden ayrıca oradan gelen veriyi temizlememiz gerekecek :')



In [ ]:
basliklar0 = soup.findAll("title")
# burada <title> tagine sahip olan her şeyi bul getir diyoruz. direkt basliklar objesini de
# print edebiliriz ama line break filan olsun, okuması kolay olsun istiyoruz, o yüzden
# aşağıya minik bi for loop yazdım.

for baslik in basliklar0:
  print(baslik)
# bakalım başlıkları görebiliyor muyuz. yis, görebiliyoruz. 
# kaosu önlemek ve çalışma alanımızı temiz tutmak adına baktıktan sonra böyle massive
# şeyler print eden line'ları comment out edebiliriz, hatta edelim çünkü 100 satır
# scroll etmek zor bir şey.

<title>"kadin surucu" - Google Haberler</title>
<title>Kadın sürücü aracıyla bariyerlere çarparak yaralandı - Haberler.com</title>
<title>Kadın sürücü fren yerine gaza basınca dükkanın içine girdi | Video - Sabah</title>
<title>Akasya Durağı'ndan etkilenen kadın sürücü taksici oldu​ - Yeni Şafak</title>
<title>Kısıtlama gününde başka bir aracın sıkıştırdığı kadın sürücü, bariyerlere çarptı - imaret haber</title>
<title>Evine giden bekçi kazaya karıştı…Kadın sürücü büyük şok yaşadı - Medyabar</title>
<title>Arabayla çarptığı kadını öldüren sürücü kaçtı - YeniÇağ</title>
<title>Balıkesir'in Edremit ilçesinde kadın sürücü ters dönen araçtan kendisi ... - Kamu3</title>
<title>Bursa'da kadın sürücü çalınan aracının peşinden metrelerce koştu - B Gazete</title>
<title>Aracıyla 20 metrelik uçurma yuvarlanan kadın sürücü öldü - Sözcü</title>
<title>Kısıtlamada 'dur' ihtarına uymayan kadın sürücü bekçiye çarptı - NTV</title>
<title>Beylikdüzü'nde kaza; Kadın sürücü yaralandı - Hürriyet</title>
<

In [ ]:
linktag = soup.findAll("link")
print(linktag)
# şimdi bir küçük <link> etiketine bakalım, lets see how bad it sux.
# ohgod bayaa kötü, bize asla content vermedi lol. "link/" olarak da denedim, still bad.
# yola <description> ile devam edeceiz meçbur.

[<link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>, <link/>]


In [ ]:
linkler0 = soup.findAll("description")
for link in linkler0:
  print(link)
# bu da epey messy bir şeyler print edecek ama we will fix that.

<description>Google Haberler</description>
<description>&lt;a href="https://www.haberler.com/kadin-surucu-bariyerlere-carparak-yaralandi-14160750-haberi/" target="_blank"&gt;Kadın sürücü aracıyla bariyerlere çarparak yaralandı&lt;/a&gt;&amp;nbsp;&amp;nbsp;&lt;font color="#6f6f6f"&gt;Haberler.com&lt;/font&gt;</description>
<description>&lt;a href="https://www.sabah.com.tr/video/yasam/kadin-surucu-fren-yerine-gaza-basinca-dukkanin-icine-girdi-video" target="_blank"&gt;Kadın sürücü fren yerine gaza basınca dükkanın içine girdi | Video&lt;/a&gt;&amp;nbsp;&amp;nbsp;&lt;font color="#6f6f6f"&gt;Sabah&lt;/font&gt;</description>
<description>&lt;a href="https://www.yenisafak.com/hayat/akasya-duragindan-etkilenen-kadin-surucu-taksici-oldu-3634559" target="_blank"&gt;Akasya Durağı'ndan etkilenen kadın sürücü taksici oldu​&lt;/a&gt;&amp;nbsp;&amp;nbsp;&lt;font color="#6f6f6f"&gt;Yeni Şafak&lt;/font&gt;</description>
<description>&lt;a href="https://www.imaret.com.tr/kisitlama-gununde-baska-bir-ara

Artık elimizde başlıklar & haber linkleri var. Yay!!! Şimdi bunları bi excelde buluşturacağız. 

## Part 3: Datamızı toparlıyoruz

##### **Başlıkları ve haber linklerini birleştirelim**
Şimdi başlıklara ve linklere ayrı ayrı eriştik, ama istiyoruz ki şık bi şekilde görelim bunları. Mesela başlık yazsın, altında da o habere giden link olsun gibi. Önce başlıklar ve linklerden oluşan bir dataframe oluşturacağız, sonra da bu dataframei bir excel dosyasına kaydedeceğiz.

**Önce dataframe oluşturuyoruz.**
Elimizde iki tane liste vardı: links0 ve basliklar0. Bunlarla çalışacağız.

In [ ]:
dataframe0 = pd.DataFrame(list(zip(basliklar0, linkler0)), 
# pd bizi pandas'ın içine götürüyor. buradan DataFrame fonksiyonunu alıyoruz.
# liste haline getirmek için list() kullanıyoruz. iki elemanı paralel iterate etmek 
# için zip() kullanıyoruz. sütunlarımız bunlar olacak.
             columns = ["Haber Başlıkları", "Linkler"])
              #sütunlara isim koyuyoruz.

print(dataframe0)
# isnt she pretty :3

                                      Haber Başlıkları                                            Linkler
0                   ["kadin surucu" - Google Haberler]                                  [Google Haberler]
1    [Kadın sürücü aracıyla bariyerlere çarparak ya...  [<a href="https://www.haberler.com/kadin-suruc...
2    [Kadın sürücü fren yerine gaza basınca dükkanı...  [<a href="https://www.sabah.com.tr/video/yasam...
3    [Akasya Durağı'ndan etkilenen kadın sürücü tak...  [<a href="https://www.yenisafak.com/hayat/akas...
4    [Kısıtlama gününde başka bir aracın sıkıştırdı...  [<a href="https://www.imaret.com.tr/kisitlama-...
..                                                 ...                                                ...
96   [Türkiye'deki kadın sürücü sayısı açıklandı - ...  [<a href="https://www.trthaber.com/haber/yasam...
97   [Kadınların trafikte yol verme kavgası karakol...  [<a href="https://www.milliyet.com.tr/gundem/k...
98   [Yerde boğuştular! Kadın sürücü yumruk yu

In [ ]:
import re
dirty_URL = pd.DataFrame.to_string(dataframe0)
clean_URL = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', dirty_URL)
print(clean_URL)

['https://www.haberler.com/kadin-surucu-bariyerlere-carparak-yaralandi-14160750-haberi/', 'https://www.sabah.com.tr/video/yasam/kadin-surucu-fren-yerine-gaza-basinca-dukkanin-icine-girdi-video', 'https://www.yenisafak.com/hayat/akasya-duragindan-etkilenen-kadin-surucu-taksici-oldu-3634559', 'https://www.imaret.com.tr/kisitlama-gununde-baska-bir-aracin-sikistirdigi-kadin-surucu-bariyerlere-carpti/83782/', 'https://medyabar.com/haber/7039589/evine-giden-bekci-kazaya-karistikadin-surucu-buyuk-sok-yasadi', 'https://www.yenicaggazetesi.com.tr/antalyada-arabayla-carptigi-kadini-olduren-surucu-kacti-454004v.htm', 'https://www.kamu3.com/balikesirin-edremit-ilcesinde-kadin-surucu-ters-donen-aractan-kendisi/304881/', 'https://www.bgazete.com.tr/haber/6910708/bursada-kadin-surucu-calinan-aracinin-pesinden-metrelerce-kostu', 'https://www.sozcu.com.tr/2021/gunun-icinden/araciyla-20-metrelik-ucurma-yuvarlanan-kadin-surucu-oldu-6338138/', 'https://www.ntv.com.tr/turkiye/kisitlamada-dur-ihtarina-uymay

RegEx ile linkleri sanırım temizleyebiliriz ama çok iyi RegEx bilmediğim için imma skip that part, excel'den elle temizleyeceğim. RegEx'ine güvenen varsa http:// ile başlayan kısmı address edecek bir şey yazabilir mesela :> O regex ile kısa bi Pandas kodu yazıp Linkler kolonunu tertemiz yapabiliriz.

**Hemen dataframe0'da olan linkleri RegEx ile temizliyoruz.**

In [ ]:
import re
dirty_URL = pd.DataFrame.to_string(dataframe0)
clean_URL = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', dirty_URL)
type(clean_URL)

list

**Şimdi de yeni data frame (df) oluşturuyoruz.**
Yeni df'de eski dirty linkler olsun istemiyoruz. O nedenle sadece başlıkları ve clean linkleri içeren yeni bir dataframe oluşturuyoruz. Buna da dataframe1 diyoruz.

In [ ]:
# Önce basliklar0 ile aynı uzunlukta olması için clean_URL'ye ilk eleman
# olarak [Google Haberler] diye bir "alt başlık(?)" ekledik.
clean_URL.insert(0, "[Google Haberler]")

# Şimdi aynı uzunlukta olduklarına göre önceki df'i (dataframe0) oluşturduğumuz şekilde
# bu sefer temiz linklerin olduğu bir df (dataframe1) oluşturuyoruz.
dataframe1 = pd.DataFrame(list(zip(basliklar0, clean_URL)), 
             columns = ["Haber Başlıkları", "Linkler"])

# Olmuş gibi gözüküyor. Haber başlıkları ve linkler aligned.
print(dataframe1)

                                      Haber Başlıkları                                            Linkler
0                   ["kadin surucu" - Google Haberler]                                  [Google Haberler]
1    [Kadın sürücü aracıyla bariyerlere çarparak ya...  https://www.haberler.com/kadin-surucu-bariyerl...
2    [Kadın sürücü fren yerine gaza basınca dükkanı...  https://www.sabah.com.tr/video/yasam/kadin-sur...
3    [Akasya Durağı'ndan etkilenen kadın sürücü tak...  https://www.yenisafak.com/hayat/akasya-duragin...
4    [Kısıtlama gününde başka bir aracın sıkıştırdı...  https://www.imaret.com.tr/kisitlama-gununde-ba...
..                                                 ...                                                ...
96   [Türkiye'deki kadın sürücü sayısı açıklandı - ...  https://www.trthaber.com/haber/yasam/turkiyede...
97   [Kadınların trafikte yol verme kavgası karakol...  https://www.milliyet.com.tr/gundem/kadinlarin-...
98   [Yerde boğuştular! Kadın sürücü yumruk yu

***Bundan sonraki kodlarda dataframe0 olanları dataframe1 ile değiştirmemiz gerekiyor.***

**Şimdi dataframe'imizden excel dosyası oluşturacağız.**
<br>
Ben bilgisayarımda bir excel dosyası oluşturdum, ismini de DataPart1.xlsx koydum. İçi boş, şimdi onu açıp içini dolduracağız.

In [ ]:
from google.colab import files
uploaded = files.upload()
#xlsx dosyasını upload ettik.

Saving aa.xlsx to aa.xlsx


In [ ]:
writer = pd.ExcelWriter('DataPart1.xlsx')
# dedik ki Panda, ExcelWriter modülü ile bizim bu dosyayı açsın; bu operasyonun ismi
# de writer olsun.

dataframe1.to_excel(writer)
# dataframe0'daki her şeyi excel dosyamıza yazıyoruz.
writer.save()
# ve kaydediyoruz. Ben bu dosyayı download edip excelde find & replace ve biraz
# da gelişmiş annotation kaslarımla linkleri temizledim. RegEx yazma işini konuşalım,
# her dataset için böyle zaman harcamayalım.

# Kapanış
Bu dosyada oluşturduğumuz Excel'i yeni bir notebook'ta açacağız. Özetle şunları yapacağız: 
* Python dosyasını yeni notebookta açacağız. Bir dataframe'e dönüştüreceğiz.
* Linkler kolonundaki linkleri tek tek alan, o sayfaları ziyaret eden, sayfalardaki `<body>` tagli metinleri çeken ve bu metinleri dataframe'e yapıştıran bir fonksiyon yazacağız. 
* Yukarıdaki fonksiyonu çalıştırıp yeni dataframe'i excel dosyamıza kaydeceğiz. Böylece elimizde haber başlığı, haber linki ve haber metninden oluşan 4 tane excel dosyası olacak. Bu dosyaları birleştirip sentiment analysis kısmını yaparız sonra da.
<br>
<br>
Şu an beynim yetmiyor ama bu bahsettiğim fonksiyon için yukarıda BeautifulSoup ile yaptığımız, findAll kullandığımız şeyleri bi loop içinde yazacağız. Would you like to give it a try? Ben de biraz dinlenince bakacağım tekrar.
<br>
<br> 
Umarım yeterince self explanatory yazmışımdır kodu ve buraya kadar bugsız errorsız gelebilmişsinizdir ✨

